In [3]:
import os
import numpy as np
import pickle
import json
import random
from tqdm import tqdm

db = "/home/yrb/code/speechbrain/data/voxceleb/vox2/spmel"
permute_rate = 20

In [ ]:
npy_list = os.listdir(db)
files = [os.path.join(db, f) for f in npy_list if f.endswith(".npy")]
# filename without extension
spkrID_list = [f[:-4] for f in npy_list if f.endswith(".npy")]
mislabel_dict = {}
for f in tqdm(files):
    data = np.load(f)
    for i in range(data.shape[0]):
        # get random float between 0 and 1
        if random.random() <= permute_rate/100:
            # remove filepath
            filename = os.path.split(f)[-1][:-4]
            key = filename + "_{}".format(i)
            # sample a random speaker
            mislabel_dict[key] = random.sample(spkrID_list, 1)[0]

In [10]:
# save to json
with open(f"/home/yrb/code/speechbrain/data/jsons/Permute/voxceleb2_{permute_rate}%_mislabel.json", "w") as f:
    json.dump(mislabel_dict, f)

# Stat file size

In [7]:
# stats of file size
def stat(db):
    files = os.listdir(db)
    size = [os.path.getsize(os.path.join(db, f))/(1024*1024) for f in files if f.endswith(".npy")]
    
    # print file size > 2MB
    return size


size = stat(db)

In [16]:
cnt = 0
sum_size = 0
for i in size:
    if i > 30:
        cnt += 1
        sum_size += i
print(cnt, sum_size)

761 29436.036499023438


# Breakdown large speakerID.npy into one utter per .npy

In [27]:
db_single = "/home/yrb/code/speechbrain/data/voxceleb/vox2/spmel_single"

npy_list = os.listdir(db)
spkrID_list = [f[:-4] for f in npy_list if f.endswith(".npy")]
files = [os.path.join(db, f) for f in npy_list if f.endswith(".npy")]

In [28]:
for spkr_i, utter_p in enumerate(tqdm(files)):
    data = np.load(utter_p)
    for utter_i in range(data.shape[0]):
        # get random float between 0 and 1
        cur_utter = data[utter_i, :, :].reshape(1, data.shape[1], data.shape[2])
        # save utter to db_single
        np.save(os.path.join(db_single, spkrID_list[spkr_i] + "_" + str(utter_i) + ".npy"), cur_utter)

100%|██████████| 5994/5994 [15:19<00:00,  6.52it/s]
